In [42]:
import subprocess
import psycopg2
import time

In [44]:
from haystack import Pipeline, Document
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack_integrations.components.generators.ollama import OllamaGenerator
from haystack_integrations.components.embedders.ollama import OllamaDocumentEmbedder
from haystack_integrations.components.embedders.ollama import OllamaTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

In [46]:
import numpy as np
import matplotlib as plt
import pandas as pd

In [48]:
DB_CONFIG = {
    "dbname": "analytics_team",
    "user": "postgres",
    "password": "pasted-sprang-siesta-shortwave",
    "host": "192.168.1.24",
    "port": "5432"
}

In [50]:
def connect_and_query(q):
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute(q)
    wr = pd.DataFrame(cursor.fetchall())
    cursor.close()
    conn.close()
    return wr    

In [52]:
table = "survey_data_DGData10"
q_tgt = "from public.\"" + table + "\""
q_op = "select"
q_cols = "\"Degree_Department\""
q_cond = ""
q = [q_op, q_cols, q_tgt]
if len(q_cond) > 0:
    q.append(q_cond)
q = " ".join(q)
print(q)

select "Degree_Department" from public."survey_data_DGData10"


In [56]:
col = connect_and_query(q)

In [58]:
print(col[0].unique())

['Department of Mathematics' 'Department of Mathematics '
 'Department of Mathematical Sciences' 'School of Mathematics'
 'School of Mathematical Sciences'
 'Mathematics, Statistics & Computer Science Department'
 'PhD Program in Mathematics' 'Department of Statistics'
 'Department of Mathematics & Statistics'
 'Courant Institute of Mathematical Sciences'
 'Department of Mathematical Science'
 'Department of Mathematics & Computer Science'
 'Department of Mathematics & Science'
 'Department of Mathematics & Statistical Sciences'
 'Mathematics & Computer Science Department'
 'Mathematics & Statistics Department' 'Mathematical Sciences Department'
 'Applied & Computational Mathematics Department'
 'Department of Statistics & Probability'
 'Department of Mathematics, Statistics & Comp Science'
 'Department of Applied Mathematics'
 'Infomation Systems Statistics & Management Science Department'
 'Department of Statistics GIDP'
 'Department of Statistics & Applied Probability'
 'Department 

In [114]:
document_store = InMemoryDocumentStore()
embedder = OllamaTextEmbedder()
responses = col[0].unique()
arr = [Document(content=x) for x in responses]
# print(arr[0].content)
result = embedder.run(text="What do llamas say once you have thanked them? No probllama!")

print(result['embedding'])
# documents_with_embeddings = embedder.run(arr)
# print(documents_with_embeddings)
# document_store.write_documents(documents_with_embeddings)

ResponseError: model "nomic-embed-text" not found, try pulling it first (status code: 404)

In [247]:
template = """
Given only the following information, structurally perform the action requested.
Ignore your own knowledge.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Action: {{ query }}?
"""

In [26]:
pipeline = Pipeline()

pipeline.add_component("text_embedder", OpenAIDocumentEmbedder())
pipeline.add_component("retriever", InMemoryEmbeddingRetriever(document_store = document_store))
pipeline.add_component("prompt_builder", PromptBuilder(template=template))
pipeline.add_component("llm", OllamaGenerator(model="llama3.1", url="http://localhost:11434"))
pipeline.connect("text_embedder", "retriever")
pipeline.connect("retriever","prompt_builder")
pipeline.connect("prompt_builder", "llm")

query = f"Create a surjective mapping of the inputs {responses} the unique departments into concise subject in format of 'Title: Subject'"
response = pipeline.run({"prompt_builder": {"query": query}, "retriever": {"query": query}})

ValueError: None of the following authentication environment variables are set: ('OPENAI_API_KEY',)

In [261]:
print(response["llm"]["replies"][0])

Based on the provided context, I will create a mapping of the departments and schools to single-subject departments. Since "Mathematics" is the most common single subject across multiple institutions, I will map all relevant departments and schools to it.

Here's the mapping:

1. Department of Mathematics and Statistics → **Mathematics**
2. Department of Mathematics → **Mathematics**
3. Department of Mathematics → **Mathematics** ( duplicate entry)
4. Department of Mathematical Sciences → **Mathematics**
5. School of Mathematics → **Mathematics**
6. School of Mathematical Sciences → **Mathematics** (note: this could also be mapped to "Statistics", but considering the other departments, I've chosen "Mathematics")
7. Mathematics, Statistics & Computer Science Department → **Mathematics** (since Computer Science is not a department in mathematics or statistics, and only one instance is provided)
8. PhD Program in Mathematics → **Mathematics**
9. Department of Statistics → **Statistics**



In [ ]:
map = {}